In [20]:
import os
import numpy as np
import pandas as pd
import wandb

In [2]:
import sys
sys.path.append('../src/')

%load_ext autoreload
%autoreload 2
# Importing our custom module(s)
import folds
import utils

In [8]:
repo_path = '/cluster/tufts/hugheslab/eharve06/bdl-transfer-learning'
experiments_path = os.path.join(repo_path, 'experiments')

In [14]:
experiment_files = [os.path.join(subdir, file) \
                    for subdir, dirs, files in os.walk(experiments_path) \
                    for file in files]

In [23]:
def load_experiment(path):
    df = pd.read_csv(path, index_col='Unnamed: 0')
    if 'lrs' in df.columns:
        df.lrs = df.lrs.apply(lambda string: np.fromstring(string[1:-1], sep=', '))
    return df

In [24]:
df = load_experiment(experiment_files[0])
df.head()

,epoch,train_loss,train_auroc,val_loss,val_auroc,test_loss,test_auroc,lrs
0,1,42634.987546,0.675034,358831.197917,0.648496,358831.364583,0.558973,"[0.03162277660168379, 0.03162277613999105, 0.0..."
1,2,42634.151493,0.712143,358830.135417,0.641702,358830.135417,0.574124,"[0.03162269857567421, 0.031622686109992255, 0...."
2,3,42634.347424,0.843284,358830.656250,0.792827,358830.718750,0.714228,"[0.03162246449841546, 0.03162244002886733, 0.0..."
3,4,42633.799118,0.901479,358829.958333,0.812434,358830.250000,0.745539,"[0.03162207437221785, 0.031622037899045045, 0...."
4,5,42634.214302,0.819427,358830.541667,0.739541,358830.604167,0.643605,"[0.03162152820093174, 0.031621479724494236, 0...."


In [34]:
file = experiment_files[-1].split('/')[-1]
#file = file.replace('=', '-')
df_table = wandb.Table(dataframe=df)
df_table_artifact = wandb.Artifact(file.replace('=', '-'), type='dataset')
df_table_artifact.add(df_table, file)
df_table_artifact.add_file(experiment_files[0])
run = wandb.init(project='bdl-transfer-learning')
run.log({file: df_table})
run.log_artifact(df_table_artifact)
run.finish()

wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


In [ ]:
# Read our CSV into a new DataFrame
df = pd.read_csv(experiment_files[0])

# Convert the DataFrame into a W&B Table
iris_table = wandb.Table(dataframe=new_iris_dataframe)

# Add the table to an Artifact to increase the row
# limit to 200000 and make it easier to reuse
iris_table_artifact = wandb.Artifact("iris_artifact", type="dataset")
iris_table_artifact.add(iris_table, "iris_table")

# log the raw csv file within an artifact to preserve our data
iris_table_artifact.add_file("iris.csv")

# Start a W&B run to log data
run = wandb.init(project="tables-walkthrough")

# Log the table to visualize with a run...
run.log({"iris": iris_table})

# and Log as an Artifact to increase the available row limit!
run.log_artifact(iris_table_artifact)

# Finish the run (useful in notebooks)
run.finish()